In [2]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation 
import pyLDAvis
import pyLDAvis.sklearn

In [3]:
stop_words=set(nltk.corpus.stopwords.words('english'))
data = pd.read_csv('phenotypes_codes.tsv', usecols=[0,1,2,3], encoding='utf-8', lineterminator='\n', sep="\t")
data.columns = data.columns.str.replace('\r', '')

data['merged_data'] = data['merged_data'].str.strip(r'\r')
data['merged_data'] = data['merged_data'].str.strip(r'\n')
data = data.replace('\\r','',regex=True)
data['merged_data'] = data['merged_data'].str.replace("'",'')
data['merged_data'] = data['merged_data'].str.replace(".",'')
data['merged_data'] = data['merged_data'].str.strip()
data['merged_data'] = data['merged_data'].str.replace("ibd ohdsi",'')
#data = data[data.id == 2]

print(data)


      id                                             name  \
0    106                                          uveitis   
1    107                                          uveitis   
2    108                                 traumatic iritis   
3    110                                     uveitis guo    
4    111                                     chronic pain   
..   ...                                              ...   
502  620                       sensorineural hearing loss   
503  621              preterm labor with preterm delivery   
504  622                                          anxiety   
505  625                              taste sense altered   
506  626  motor neuropathy with multiple conduction block   

                                      icd10_list_codes  \
0    ['icd10_a18.53', 'icd10_a18.54', 'icd10_a50.32...   
1    ['icd10_a18.53', 'icd10_a18.54', 'icd10_a50.32...   
2    ['icd10_h20.0', 'icd10_h20.00', 'icd10_h20.04'...   
3    ['icd10_a18.53', 'icd10_a18.54

/tmp/ipykernel_1101/436090915.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['merged_data'] = data['merged_data'].str.replace(".",'')


In [4]:
tfidfvectoriser=TfidfVectorizer(max_features=1000, stop_words=stop_words)
#tfidfvectoriser.fit(data.merged_data)
tfidf_vectors=tfidfvectoriser.fit_transform(data.merged_data)
lda_model=LatentDirichletAllocation(n_components=11, learning_method='online',random_state=42,max_iter=30) 
lda_top=lda_model.fit_transform(tfidf_vectors)
LDAvis_prepared = pyLDAvis.sklearn.prepare(lda_model, tfidf_vectors, tfidfvectoriser)
pyLDAvis.display(LDAvis_prepared)

/home/ubuntu/env/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/home/ubuntu/env/lib/python3.10/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/home/ubuntu/env/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/home/ubuntu/env/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; s

In [5]:
topic_words = lda_model.components_
vocab_tfidf = tfidfvectoriser.get_feature_names()
n_top_30 = 30

for i,topic_dist in enumerate(topic_words):
     sorted_topic_dist = np.argsort(topic_dist)
     topic_words = np.array(vocab_tfidf)[sorted_topic_dist]
     topic_words = topic_words[:-n_top_30:-1]
     print("Topic", str(i+1), topic_words)

Topic 1 ['disease' 'hepatitis' 'icd10_b182' 'icd10_b1920' 'icd10_b192' 'viral'
 'diabetes' 'mellitus' 'type' 'chronic' 'icd10_n186' 'icd10_e1137'
 'icd10_e1137x9' 'icd10_e1137x1' 'icd10_e1137x2' 'icd10_e1137x3'
 'icd10_e1122' 'icd10_e113541' 'icd10_e113531' 'icd10_e113529'
 'icd10_e11353' 'icd10_e113533' 'icd10_e113532' 'icd10_e113539'
 'icd10_e11354' 'pulmonary' 'embolism' 'malignant' 'acute']
Topic 2 ['failure' 'icd10_k9183' 'icd10_k767' 'arthritis' 'icd10_m0569'
 'icd10_m06241' 'icd10_m069' 'icd10_m0622' 'icd10_m06251' 'icd10_m06221'
 'icd10_m0624' 'icd10_m0621' 'icd10_m0689' 'icd10_m06219' 'icd10_m06211'
 'icd10_m06229' 'icd10_m06242' 'icd10_m06262' 'icd10_m056' 'icd10_m068a'
 'icd10_m06261' 'icd10_m06' 'icd10_m0620' 'icd10_m0560' 'icd10_m06232'
 'icd10_m06239' 'icd10_m0626' 'icd10_m06259' 'icd10_m06252']
Topic 3 ['chronic' 'icd10_p2912' 'icd10_c9161' 'icd10_c916' 'icd10_c9162'
 'icd10_c9160' 'icd10_c9110' 'icd10_c911' 'icd10_c9112' 'disease'
 'icd10_g40509' 'icd10_g40814' 'icd10_g

/home/ubuntu/env/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [6]:
doc_topic = lda_model.transform(tfidf_vectors)
topic_list = []
for n in range(doc_topic.shape[0]):
     topic_doc = doc_topic[n].argmax() + 1
     #fO.write("Document ", n+1, "---Topic: ", topic_doc)
     topic_list.append(topic_doc)
data['topic_number'] = topic_list

data_grouped = data.groupby(['topic_number','name'])
data_grouped.sum().reset_index().to_csv('test.csv', sep="\t", index = False)
